# 🌈 Actions

There are number of methods or actions that any `Rainbow` object can do. By learning the vocabulary of just a few of these methods, you can build up some fairly complicated stories for working with data. In general, most of these actions return a `Rainbow` object that has been modified in one way or another, so you can keep adding actions after actions after actions. To show how these work, we'll create a simulated rainbow object and try a few:

In [ ]:
from chromatic import *

In [ ]:
r = SimulatedRainbow(R=100, dt=1 * u.minute).inject_transit().inject_noise()

## `🌈.bin()`
While we should generally try to avoid fitting to binned data when possible, there will often be times where it's helpful to bin to particular grid of wavelengths and/or times. You can do this using the `.bin()` function. We'll create a simulated rainbow object and use it to show how this works.

In [ ]:
def summarize(x):
    print(
        f"""
    {x} is a {type(x).__name__}.
    It has a {x.nwave} wavelengths and {x.ntime} times. 

    Its 5 first wavelengths:{np.round(x.wavelength[:5], 2)}
    Its 5 first times:{np.round(x.time[:5], 2)}
    """
    )

In [ ]:
summarize(r)

To bin in **wavelength**, it can take the following inputs:
- `dw=` to bin in wavelength to a particular $d\lambda$ width. This will create a linear grid in wavelength.
- `R=` to bin in wavelength to particular $R = \lambda/d\lambda$. This will create a logarithmic grid in wavelength.
- `wavelength=` to bin to any custom wavelength grid. *(Right now, this tries to make a good guess for the edges of the bins based on their centers; we should probably implement a more straightfoward way to insert and track specifc left and right bin edges.)*

In [ ]:
b = r.bin(dw=0.5 * u.micron)
summarize(b)

In [ ]:
b = r.bin(R=10)
summarize(b)

In [ ]:
b = r.bin(wavelength=np.linspace(1, 2, 4) * u.micron)
summarize(b)

To bin in **time**, it can take the following inputs:
- `dt=` to bin in time to a particular $dt$ width. This will create a linear grid in time.
- `time=` to bin to any custom time grid. *(Right now, this tries to make a good guess for the edges of the bins based on their centers; we should probably implement a more straightfoward way to insert and track specifc left and right bin edges.)*

In [ ]:
b = r.bin(dt=0.25 * u.hour)
summarize(b)

In [ ]:
b = r.bin(time=np.linspace(-1, 1, 5) * u.hour)
summarize(b)

And of course, you can combine to bin in both wavelength and time in the same step.

In [ ]:
b = r.bin(dw=100 * u.nm, dt=10 * u.minute)
summarize(b)

In [ ]:
fi, ax = plt.subplots(2, 1, sharex=True, figsize=(6, 6))
r.imshow(ax=ax[0], vmin=0.98, vmax=1.02)
plt.title("Unbinned")
plt.xlabel("")
b.imshow(ax=ax[1], vmin=0.98, vmax=1.02)
plt.title("Binned");

## `🌈.normalize()`
If you're starting in units of photons detected at your telescope per wavelength, you may want to normalize a Rainbow by dividing through by its median spectrum. That's all that the `.normalize()` action does. 

In [ ]:
r.normalize().imshow();

In some cases, you might also be curious to divide by the median light curve, to look for small variations away from the overall transit shape. You can customize whether you want to normalize in wavelength or time by suppyling the `axis=` keyword when you call the `.normalize()` function. (It defaults to `axis='wavelength'`.)

In [ ]:
r.normalize(axis="time").imshow();

(In this case, there were not transit depth or limb-darkening variations injected into the simulated transit, so normalizing through time entirely erases any hint of the transit.)

## `🌈.inject_noise()`

We can inject noise, for example to simulate the observing the same system at a greater distance or with a smaller telescope. The simplest way to set the noise level is with the `signal_to_noise` keyword argument.

In [ ]:
with_noise = SimulatedRainbow().inject_noise()
with_noise.imshow();

## `🌈.inject_transit()`

For simulations, it's really useful to have a way to inject a simulated transit into a chromatic light curve. We've already seen this function in action above. 

In [ ]:
with_transit = SimulatedRainbow().inject_transit()
with_transit.imshow();

You can customize some of the model parameters. See the docstring for more details, but the most basic would be to provide an array of planet radii corresponding to each wavelength.

In [ ]:
with_transit = SimulatedRainbow().inject_transit(
    planet_radius=np.linspace(0.2, 0.1, noise.nwave)
)
with_transit.imshow();

## `🌈.inject_systematics()`

High-precision observations of transiting exoplanet often encounter systematic noise sources that are correlated in complicated ways in time or wavelength or with various hidden quantities. You can inject a cartoon version of these systematics (a linear combination of real or imagined time-like, wave-like, and/or flux-like quantities). See its docstring for details.

In [ ]:
with_systematics = SimulatedRainbow().inject_systematics()
with_systematics.imshow();

Of course, these commands can be linked together to create a semi-realistic transit dataset.

In [ ]:
(SimulatedRainbow().inject_transit().inject_systematics().inject_noise().imshow());

## `🌈 + 🌈`

You can perform mathematical operations on Rainbow objects. For now, this effectively just does math on the flux arrays. 

In [ ]:
a = SimulatedRainbow().inject_noise()
b = SimulatedRainbow().inject_noise()

In [ ]:
(a + b).imshow();

In [ ]:
(a - b).imshow();

In [ ]:
(a * b).imshow();

In [ ]:
(a / b).imshow();

## `🌈[:,:]`

You can trim a Rainbow in wavelength (first dimension) and/or time (second dimension) by slicing it in a similar way that you might to any other 2D array. 

In [ ]:
# the original array
summarize(r)

In [ ]:
# using slices
summarize(r[5:10, ::2])

In [ ]:
# using indices
i_wavelengths = np.arange(5, 10)
i_times = np.arange(0, r.ntime, 2)
summarize(r[i_wavelengths, i_times])

In [ ]:
# using boolean masks
ok_wavelengths = r.wavelength < 2 * u.micron
ok_times = np.abs(r.time) < 1 * u.hour
summarize(r[ok_wavelengths, ok_times])

# Viewing a 🌈's History

Most actions that return `Rainbow` objects be recorded in that object's `metadata['history']` entry. This is meant to be an approximate summary of the steps that led up to the creation of the current 🌈. You can view this history by calling the `.history()` method.

In [ ]:
x = (
    SimulatedRainbow()
    .inject_noise()
    .inject_transit()
    .bin(R=5, dt=5 * u.minute)
    .normalize()
)
x.history()

By default, it gives an almost copy-paste-able string version of the history, but you can also access this history as a table.

In [ ]:
x.history("table")